## Part 1: Building String Scoring Models ##

__Reference:__ Some code snippets for the initial part (pre-processing etc.) are taken from the blog mentioned in the Project description file. [https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py](https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py)

The aim of this blog is to build string scoring models which will then be combined into a single language detector.

The scoring models are recurrent neural networks (RNNs), using ***Long-Short Term Memory (LSTM)*** units. LSTMs are particularly well suited for learning patterns in long input sequences (Ex: text, speech), without being affected by the vanishing gradient problem that plagues deep neural networks. **Given a sequence of input characters**, these neural networks are trained to **predict the next character** in that sequence.

These models are then used to calculate the **likelihood** that the language of a test sequence is the same as the language on which the model is trained. For a **binary classification** problem, the **difference of likelihoods** is used as the decision value, whereas the **maximum likelihood** is used as the decision function for **multi-language models**.

For the extra credit part of this assignment, we have attempted a few experiments, including improving the binary models' accuracy and implementing a multi-language classification model. These experiments and their results are documented and presented in separate notebooks.

### Importing Libraries and Modules ###

In [ ]:
from keras.layers import Activation, Dense, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.optimizers import RMSprop
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import sys
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from itertools import cycle

### Setting Model Parameters ###

In [ ]:
sys.setrecursionlimit(10000)     # setting this higher in case a saved model is loaded
RANDOM_STATE = 100
np.random.seed(100)              # set seed for reproducibility
tf.random.set_seed(100)          # set seed for reproducibility

val_frac = 0.2                   # fraction of data to be set as validation set
test_frac = 0.2                  # fraction of data to be set as test set

LANGUAGES = ['eng', 'frn']       # list of languages for classification

maxlen = 40                      # length of sliding window (input sequence) to the LSTM model
step = 1                         # the step size for cutting the input sequence into redundant sequences of size maxlen

len_test = 5                     # length of the test substrings
num_test = 100                   # number of test substrings taken from ONE particular language

EPOCHS = 5                       # number of epochs

### Define methods to read and featurize the text ###
The text is split into overlapping sentences of fixed length using a sliding window. A charset containing all the unique characters across all languages is used for each language.

In [ ]:
def read_file(lang):
    
    '''
    This function takes a language name as input, reads the particular contents of the language file from the folder 'subset',
    converts the text to lower case and makes a list of the unique characters encountered. The text, as a string, and the unique
    characters, as a list, are returned by the function.
    '''
    
    # path where the language text files are stored
    path = './subset/{}.txt'.format(lang)

    text = open(path).read().lower()
    print('\n\n{} corpus length: {}'.format(lang, len(text)))
    
    # store the unique characters as a list
    chars = sorted(list(set(text)))
    print('unique chars:', len(chars))
    
    return text, chars

In [ ]:
def cut_sequences(text):
    '''
    This function recieves a string as input, and cuts it intoe semi-redundant sequences of maxlen characters each. Basically
    sliding a window of length 'maxlen' over the text, and taking increments of 'step' at a time. These sequences are stored in
    a list called 'senteces'. The last character of each sequence is stored in a list called 'next_chars'. The function returns
    the two lists, 'sentences' and 'next_chars'.
    '''
    sentences = []
    next_chars = []
    
    for i in range(0, len(text) - maxlen, step):
        sentences.append(text[i: i + maxlen])
        next_chars.append(text[i + maxlen])
    
    print('nb sequences:', len(sentences))
    
    return sentences, next_chars

In [ ]:
def vectorize(charset, sentences, next_chars):
    '''
    This function takes as input a characterset (list of unique characters), sentences (list of redundant character sequences) 
    and next_chars (list of the last character of each sequence in senetences). Then it generates the vector reprsentation of
    sentences and next_chars, X and y respectively.
    '''
    
    print('Vectorizing text...')
    
    # X: 3 dimensional, dimension 1: number of sequences (number of inputs), dimension 2: the length of the sliding
    # window, maxlen, dimension 3: length of the characterset. So for a given sequence or input, dimension 2 represents 
    # the different characters in the sequence and for each character only one value in the charset is set to 1 (1-hot encoding)
    
    X = np.zeros((len(sentences), maxlen, len(charset)), dtype=np.bool)
    
    
    # Y: 2 dimensional, dimension 1: number of inputs (each input is a single chracter which is the last character in
    # the corresponding sequence contained in X). The second dimension is the character set representing the 1-hot encoding
    # scheme for the character
    
    y = np.zeros((len(sentences), len(charset)), dtype=np.bool)
    
    # convert the characters to dictinary for easy access and for 1-hot encoding
    char_indices = dict((c, i) for i, c in enumerate(charset))
    
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    
    return X, y

### Define basic methods for building the models
The data is randomly split into a train set (80%) and a holdout test set (20%). The model is trained only on the train set, i.e. it never sees any samples from the holdout set during training.  

The neural network models for each language are saved after training, so that they can be loaded later for reuse without retraining.

In [ ]:
def split_n_save(X, y, charset, lang):
    '''
    Takes input matrix X, labels y, the characterset and language name. Splits X, y into the training and test sets, stores
    them in a folder called 'splits' and returns the splits as well.
    '''
    
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=test_frac, random_state=RANDOM_STATE)
    
    np.savez_compressed('./splits/{}.npz'.format(lang),
                        xtrain=X_train,
                        xtest=X_test,
                        ytrain=y_train,
                        ytest=y_test,
                       charset=charset)
    
    return X_train, X_test, y_train, y_test

In [ ]:
def build_model(lang, charset):
    '''
    Builds and returns a single layered LSTM model, with size 128. The input shape is a tuple (maxlen, lenght of the 
    characterset). maxlen is the string sequence lenght and the charset will contain the probabilities of each character 
    in the sequence. The activation used for the output is 'softmax' to output normalized probability values for each character 
    in the set.
    '''
    
    model = Sequential(name=lang)
    
    # add an LSTM layer
    model.add(LSTM(128, input_shape=(maxlen, len(charset))))
    
    # add a dense layer with size as the length of the characterset
    model.add(Dense(len(charset)))
    
    # softmax activation
    model.add(Activation('softmax'))
    
    optimizer = RMSprop(learning_rate=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    
    return model

In [ ]:
def normalize_preds(preds, temperature=1.0):
    '''
    This function takes predictions and normalizes them so that they sum to 1. In this notebook, this function is actually
    redundant as the predictions are already normalized values.
    '''
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    
    return preds

In [ ]:
def train_n_save(model, X, y):
    '''
    Trains and saves a model on the input X and output y. The number of epochs and fraction of data used for validation are
    taken from the model parameters set in the initial code blocks at the start of the notebook
    '''

    model.fit(X, y, batch_size=128, epochs=EPOCHS, verbose=2, validation_split=val_frac)
    
    model.save('./models/{}.h5'.format(model.name))

### Train the neural networks
Using the methods defined above, we now read the data, split it and train the neural network models using the train split.

In [ ]:
# read the language files and store all unique characters in a common characterset
charset = set()
for lang in LANGUAGES:
    text, chars = read_file(lang)
    charset = charset.union(chars)

# for each language
for lang in LANGUAGES:
    
    # get the text of the language
    text, _ = read_file(lang)
    
    # cut it into semi redundant character sequences, and also get the last character of each sequence
    sentences, next_chars = cut_sequences(text)
    
    # vectorize sentences and next_chars
    X, y = vectorize(charset, sentences, next_chars)
    
    # get the training and test splits
    X_train, X_test, y_train, y_test = split_n_save(X, y, charset, lang)
    
    # train and save the language model
    model = build_model(lang, charset)
    train_n_save(model, X_train, y_train)

### Load the models, and calculate log likelihoods for the test strings
100 substrings of length 5 are randomly sampled from the holdout sets of each language, and these strings are used as the test sequences for the neural network models.

In [ ]:
def load_the_model(lang):
    '''
    Given a language name, load the model and its associated data and return them
    '''
    
    model = load_model('./models/{}.h5'.format(lang))
    
    data = np.load('./splits/{}.npz'.format(lang))
    
    X_train = data['xtrain']
    X_test = data['xtest']
    y_train = data['ytrain']
    y_test = data['ytest']
    
    return model, X_train, X_test, y_train, y_test

In [ ]:
def select_subs(X_test): 
    '''
    Given the test sequence, randomly select and return num_test(100) number of len_test(5) character substrings
    '''
    test = np.zeros((num_test, len_test, len(charset)), dtype=np.bool)
    
    # randomly select num_test sequences
    lines = np.random.choice(len(X_test), num_test, replace = False)
    
    # from each randomly selected sequence, randomly select a continuous sequence of len_test substrings
    for counter, line in enumerate(lines):
        
        # random substring selection
        i = np.random.choice(X_test.shape[1] - len_test)
        test[counter,:,:] = X_test[line,i:i+len_test,:]
        
    return test

In [ ]:
# Create the test substring dataset and the the true labels

test = np.empty((0, len_test, len(charset)), dtype = np.bool)

# one hot encoding for the labels
labels_test = np.zeros((num_test*len(LANGUAGES), len(LANGUAGES)), dtype = np.int64)    

for i, lang in enumerate(LANGUAGES):
    data = np.load('./splits/{}.npz'.format(lang))
    test = np.vstack((test,select_subs(data['xtest'])))
    labels_test[(i*num_test):(i+1)*num_test, i] = 1.0

In [ ]:
def load_all_models():
    '''
    Load all models and return as a list
    '''
    
    models = []
    for count, lang in enumerate(LANGUAGES, start = 1):
        print("loading model no: {}".format(count))
        models.append(load_the_model(lang)[0])
    return models

### Make the predictions
Initially, each model is seeded with an unbiased start sequence (a sequence in which every position is equally likely to be any character from the entire charset). The probability of the first letters of the test sequences are then found.

We go on to predict the probability of the next characters of the test sequences given the previous characters, one at a time.

The final predicted value (y_hat) is the difference of log probabilities of the english and french models.
$$
y_{hat} = log[Pr(string | eng)] - log[Pr(string | frn)]
$$

In [ ]:
def predict(test, model):
    '''
    Function takes a test set containing substrings of fixed length in different languages and a language specific model. The
    model then predicts the likelihood of each substring belonging to that specific language. 
    '''
    
    num_samples = test.shape[0]

    likelihoods = np.zeros(num_samples, dtype=np.float64)

    start_shape = (num_samples, maxlen, len(charset))

    # here we set the probabilities of all characters in start sequence to be equal
    # essentialy for the first character to be predicted the model has no context
    start_value = 1.0 / len(charset)
    start = np.full(start_shape, start_value, dtype=np.float64)

    test_sequences = start

    # enumerate over characters of all test substrings and predict their likelihoods
    for i, chars in enumerate(zip(*test)):
        preds = model.predict(test_sequences, verbose=0)
        preds = normalize_preds(preds)

        # calculate the log probability
        likelihoods += np.log(preds[test[:, i, :]])

        # move the sliding input sequence by appending the test character from the substring
        next_chars = np.reshape(chars, (test_sequences.shape[0], 1, test_sequences.shape[2]))
        test_sequences = np.concatenate((test_sequences[:, 1:, :], next_chars), axis=1)

    return likelihoods

### Plot the Reciever Operating Characteristics (ROC) curve
The ROC curve shows the true positive rate vs the false positive rate for a binary classification task. Both the neural network models are fed with the same test sequences, and the predicted likelihoods are used to plot the ROC curve using matplotlib and sklearn.

In [ ]:
def accuracy_and_roc():
    '''
    Calcluate the accuracy and plot the ROC
    '''
    
    # get all models as a list
    models = load_all_models()
    
    # get the likelihood for each model
    likelihoods = np.zeros((len(test), len(models)), dtype = np.float64)
    for i, model in enumerate(models):
        likelihoods[:,i] = predict(test, model)
    
    
    # calculate the accuracy
    count = 0
    for i in range(len(test)):
        if( (likelihoods[i,0] > likelihoods[i,1]) == bool(labels_test[i,0]) ): # correct prediction
            count += 1
    accuracy = (count/len(labels_test))*100.0
    
    # plot the ROC on semilogx
    fpr, tpr, thresholds = roc_curve(labels_test[:,0], likelihoods[:,0] - likelihoods[:,1])
    roc_auc = auc(fpr, tpr)
    
    
    print ("\nAccuracy: {:0.3f}, Area under the curve: {:0.3f}".format(accuracy, roc_auc))
    
    plt.plot(fpr, tpr, lw=2, color='blue', linestyle = 'solid', label='area={:0.3f}'.format(roc_auc))
    plt.plot([0, 1], [0, 1], linestyle='dashed', lw=2, color='k', label='Luck')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristics')
    plt.legend(loc="lower right")
    plt.show()  
    
    return (accuracy, roc_auc)

In [ ]:
# Calculate and print the accuracy, ROC and area under the curve
accuracy, area = accuracy_and_roc()

## Part 1: Questions ##

### Q1. Is this model good ? ###
  
_Ans:_  Given the small amount of training data and the small size of the test substring the accuracy of the model and the area under the curve seem pretty good. Intutively, if the length of the substring is increased from 5, the accuracy should increase since more language specific semantics, structural and contextual information will be captured in a longer substring which will help in classfication. As a test, when we increased the length of the substring to 20 from 5, the accuracy and the area under the curve both shot up to 96% and 0.97 respectively reaching almost perfect classfication


### Q2. What are at least three alternatives to language detection that you can think of or find on the internet? What are the pros and cons of each approach? ###

_Ans: Three alternatives are:_
  
  a) Language detection using N-grams: Based on creating a language profile of N-gram frequencies (N-gram matrix) using the training document. The test text is then converted to an N-gram profile using similar technique and the distance between the test text and all language profiles is calculated. Whichever distance is the least is the most probable language.
  
  Pros:
  
  * Simple and easy to understand, not very computationally heavy (at least as compared to LSTM)
  
  Cons:
  
  * Since its based on language N-gram profiles, it requires a lot of preprocessing and is unable to use contextual informaiton in the language. For example, whereas LSTMS can learn complex language structures, like quotation marks etc, the N-gram model cannot
  
  b) Bag of Words (_or frequent words counting_): Make an extensive collection of words (dictionary) for each language. Then, given a text, classify based on the frequency of words in the text.
 
  Pros:
  
  * Relatively simple with few hyperparameters. For example, N-grams has a number of parameters, like, the length of the N-gram sequence and the overlap in the sequences.
  
  Cons:
  
  * If a word in the test set was not in the training set, then bag of words has no interpretation for that word. On the other hand, an N-gram scheme can still have some insight about a word based on the N-gram sequence in that word
  
  * It needs large storage space to store an entire language dictionary of all possible words and their forms (eg. fast, faster, fastest, etc.)
  
  * Susceptible to spelling errors.
  
  c) Markov Models for Language Identification: The occurrences of letters in a word is regarded as a stochastic process and the word is represented as a Markov chain where letters are states.
 
  Pros:
  
  * Compared to the N-grams method, Markov chain-based system is much faster for identification since the number of letters is smaller than the number of N-grams (say tri-grams) in any language.
  
  Cons:
  
  * Language structure is not considered
  
  d) Small words technique: Similar to N-grams (tri-grams) technique, the difference being that instead of high frequency tri-grams (which can be high in number), only frequently occuring common short words (like _an, the , and_ etc.) are considered.
 
  Pros:
  
  * Compared to the tri-grams method, the small words technique is faster since for a given text input, there are less words than tr-grams and hence the sentence probability calculation is faster.
  
  Cons:
  
  * Accuracy is low for short texts and sentences. This is because in shorter senteces, there is a greater chance that no language characteristic words are used.
  

### Q3. Describe 5 ways to improve the model ###

_Ans: 5 ways to improve the model are:_
  
  1) __Hyperparamter Tuning:__ There are a number of hyperparameters in this model, (number of LSTM layers, size of layers, dropout, length of input sequence, step size etc.) which can be optimized to give better results ---> Advantage: Improvement in classfication accuracy at the cost of increased computation cost during hyperparameter optimization.
  
  2) __Early Stopping:__ Optmize the number of epochs by setting validation loss decrease criterion. For example, monitor validation loss and if the validation loss decreases by less than a set threshold (say 1%), then stop training. ---> Advantage: More efficient training in terms of time required, i.e, one does not always have to run a fixed number of epochs. Also, this method avoids overfitting by interrupting model fitting when validation loss starts to increase.
  
  3) __Diverse Inputs:__ Feeding the model, diverse styles and topics from each language under consideration should increase accuracy. The more diverse the input, like poetry, scientific texts etc. and more the type of topics covered, like sports, technology etc. the more language specific content the model will learn and will be able to generalize well.This will, though, increase computation cost at training time because of the increased training size.
  
  4) __Increase Input Sequence Length:__ The longer the input sequence, the more language specific structural elements the model can learn. For example, if the input is source code, and if the input sequence is long enough, then the model can even learn the syntax, for example, when and what type of braces to open and close etc. Obvipously this will come at the cost of larger computation complexity.
  
  5) __Parallel Processing:__ Use of GPUs, for example, will decrease the time required to train the models and the increased computational power can allow the modeler to explore more in-depth hyperparameter tuning ultimately leading to a more accurate model.